In [1]:
import numpy as np
import pandas as pd
from amplpy import ampl_notebook

ampl = ampl_notebook(license_uuid="...")

Licensed to AMPL Personal Prototyping License for <jburgy@gmail.com>.


In [2]:
%%ampl_eval
set stocks;
set accounts = {1..2};
set sectors;
# see https://groups.google.com/g/ampl/c/ULGck_3EQOM/m/yHvDLokzBQAJ
set stocks_in_sector {sectors} within stocks;

param base_rate {accounts};
param market_value {stocks};
param sign {stock in stocks} = if market_value[stock] >= 0 then 1 else -1;
param volume {stocks};

var x {stock in stocks, accounts}
          >= min(market_value[stock], 0),<= max(market_value[stock], 0);
var gmv >= 0;
var sector_mv {sectors};
var sector_nmv {sectors} >= 0;

subject to complete {stock in stocks}:
          sum {account in accounts} x[stock, account] = market_value[stock];

subject to gmv_helper:
          sum {s in stocks} sign[s] * x[s, 1] = gmv;

subject to sector_mv_def {sector in sectors}:
          sum {stock in stocks_in_sector[sector]} x[stock, 1] = sector_mv[sector];

subject to sector_nmv_pos {sector in sectors}:
          sector_mv[sector] <= sector_nmv[sector];

subject to sector_nmv_neg {sector in sectors}:
          -sector_mv[sector] <= sector_nmv[sector];

minimize total_cost:
          sum {stock in stocks, account in accounts}
          base_rate[account] * sign[stock] * x[stock, account]
          + sum {stock in stocks}
          0.2 * max(sign[stock] * x[stock, 1] - volume[stock], 0)
          + sum {stock in stocks}
          0.1 * max(sign[stock] * x[stock, 1] - 0.05 * gmv, 0)
          + sum {sector in sectors}
          0.2 * max(sector_nmv[sector] - 0.2 * gmv, 0);

In [3]:
π = pd.read_csv(
    "https://raw.githubusercontent.com/jburgy/jupyter"
    "/refs/heads/main/content/data/portfolio.csv",
    index_col=0,
)
# Fill-in blank ticker (ESC GCI LIBERTY INC SR COMMON STOCK)
π.rename(index={np.nan: "DUMMY"}, inplace=True)
π["market_value"] = π["Shares"] * π["Price"]
π["volume"] = π["Volume"] * π["Price"]

ampl.set_data(π[["market_value", "volume"]], "stocks")
ampl.param["base_rate"] = {1: 0.05, 2: 0.06}

stocks_in_sector = π.groupby("Sector").groups
ampl.set["sectors"] = stocks_in_sector.keys()
ampl.set["stocks_in_sector"] = stocks_in_sector

ampl.option["solver"] = "highs"
ampl.option["highs_options"] = {"outlev": 1}
ampl.solve()
assert ampl.solve_result == "solved"

HiGHS 1.11.0:   tech:outlev = 1

AMPL MP initial flat model has 339 variables (0 integer, 0 binary);
Objectives: 1 linear; 
Constraints:  355 linear;
Algebraic expressions:  170 max;

AMPL MP final model has 839 variables (0 integer, 0 binary);
Objectives: 1 linear; 
Constraints:  525 linear;


Running HiGHS 1.11.0 (git hash: 364c83a51): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 525 rows; 839 cols; 1529 nonzeros
Coefficient ranges:
  Matrix [5e-02, 1e+00]
  Cost   [5e-02, 2e-01]
  Bound  [1e+03, 3e+09]
  RHS    [1e+03, 6e+05]
Presolving model
194 rows, 348 cols, 867 nonzeros  0s
Dependent equations search running on 5 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
187 rows, 341 cols, 865 nonzeros  0s
Presolve : Reductions: rows 187(-338); columns 341(-498); elements 865(-664)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)

In [4]:
x = ampl.get_variable("x").get_values().to_pandas()
π.assign(
    account_1=x.loc[(π.index, 1), "x.val"].to_numpy(),
    account_2=x.loc[(π.index, 2), "x.val"].to_numpy(),
)

,Shares,Volume,Sector,Price,market_value,volume,account_1,account_2
APP,3245,1410453,Technology,163.6200,530946.900,2.307783e+08,530946.900,0.000
JWN,21589,222710,Consumer Discretionary,23.2300,501512.470,5.173553e+06,501512.470,0.000
PATH,38508,3786547,Technology,12.0415,463694.082,4.559571e+07,463694.082,0.000
PLTK,59855,56984,Technology,7.7750,465372.625,4.430506e+05,443050.600,22322.025
PLTR,10314,14703373,Technology,43.3640,447256.296,6.375971e+08,447256.296,0.000
...,...,...,...,...,...,...,...,...
APA,-19341,586971,Energy,24.7000,-477722.700,1.449818e+07,-477722.700,0.000
DV,-29507,335621,Technology,16.2850,-480521.495,5.465588e+06,-480521.495,0.000
S,-19226,1192356,Technology,25.1500,-483533.900,2.998775e+07,-483533.900,0.000
PR,-37754,1921815,Energy,13.7940,-520778.676,2.650952e+07,-520778.676,0.000
